In [1]:
import torch 
from torch import nn
import numpy as np
import random as rnd
import matplotlib
import time
import copy as copy
# Set the backend BEFORE importing pyplot
matplotlib.use('Qt5Agg')  # Or 'Qt5Agg', 'WXAgg', etc.

import matplotlib.pyplot as plt


device = "cuda" if torch.cuda.is_available() else "cpu"
device

device = "cpu"

#Tabuleiro = Board().to(device)

In [2]:
m = 6
n = 7

def rnd_N(n):
    return(int(rnd.random()*n))

def remove_from_tensor(Tensor, element):
    mask = Tensor != element
    Tensor_after_remove = Tensor[mask]
    return(Tensor_after_remove)



def Check_Conections(positionX, positionY, S): # positionY é de cima pra baixo
    List_Connections = [] # Formato conexão: [X, Y, n], n = 1,2,3,4 para leste, sudeste, sul, sudoeste
    if positionX < n-3:
        if S[positionY][positionX] == S[positionY][positionX+1] == S[positionY][positionX+2] == S[positionY][positionX+3]:
            List_Connections = List_Connections + [[positionX, positionY, 1]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY, 1]])), dim=0)
    if positionX > 2:
        if S[positionY][positionX] == S[positionY][positionX-1] == S[positionY][positionX-2] == S[positionY][positionX-3]:
            List_Connections = List_Connections + [[positionX-3, positionY, 1]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-3, positionY, 1]])), dim=0)
    if positionY < m-3:
        if S[positionY][positionX] == S[positionY+1][positionX] == S[positionY+2][positionX] == S[positionY+3][positionX]:
            List_Connections = List_Connections + [[positionX, positionY, 3]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY, 3]])), dim=0)
    if positionY > 2:
        if S[positionY][positionX] == S[positionY-1][positionX] == S[positionY-2][positionX] == S[positionY-3][positionX]:
            List_Connections = List_Connections + [[positionX, positionY-3, 3]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY-3, 3]])), dim=0)

    if positionX < n-3 and positionY < m-3:
        if S[positionY][positionX] == S[positionY+1][positionX+1] == S[positionY+2][positionX+2] == S[positionY+3][positionX+3]:
            List_Connections = List_Connections + [[positionX, positionY, 2]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY, 2]])), dim=0)
    if positionX < n-3 and positionY > 2:
        if S[positionY][positionX] == S[positionY-1][positionX+1] == S[positionY-2][positionX+2] == S[positionY-3][positionX+3]:
            List_Connections = List_Connections + [[positionX+3, positionY-3, 4]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX+3, positionY-3, 4]])), dim=0)
    if positionX > 2 and positionY < m-3:
        if S[positionY][positionX] == S[positionY+1][positionX-1] == S[positionY+2][positionX-2] == S[positionY+3][positionX-3]:
            List_Connections = List_Connections + [[positionX, positionY, 4]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY, 4]])), dim=0)
    if positionX > 2 and positionY > 2:
        if S[positionY][positionX] == S[positionY-1][positionX-1] == S[positionY-2][positionX-2] == S[positionY-3][positionX-3]:
            List_Connections = List_Connections + [[positionX-3, positionY-3, 2]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-3, positionY-3, 2]])), dim=0)

    if n-2 > positionX > 0:
        if S[positionY][positionX] == S[positionY][positionX-1] == S[positionY][positionX+1] == S[positionY][positionX+2]:
            List_Connections = List_Connections + [[positionX-1, positionY, 1]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-1, positionY, 1]])), dim=0)
    if m-2 > positionY > 0:
        if S[positionY][positionX] == S[positionY-1][positionX] == S[positionY+1][positionX] == S[positionY+2][positionX]:
            List_Connections = List_Connections + [[positionX, positionY-1, 3]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY-1, 3]])), dim=0)
    if n-2 > positionX > 0 and m-2 > positionY > 0:
        if S[positionY][positionX] == S[positionY-1][positionX-1] == S[positionY+1][positionX+1] == S[positionY+2][positionX+2]:
            List_Connections = List_Connections + [[positionX-1, positionY-1, 2]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-1, positionY-1, 2]])), dim=0)
    if n-1 > positionX > 1 and m-2 > positionY > 0:
        if S[positionY][positionX] == S[positionY-1][positionX+1] == S[positionY+1][positionX-1] == S[positionY+2][positionX-2]:
            List_Connections = List_Connections + [[positionX+1, positionY-1, 4]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX+1, positionY-1, 4]])), dim=0)

    if n-1 > positionX > 1:
        if S[positionY][positionX] == S[positionY][positionX-1] == S[positionY][positionX-2] == S[positionY][positionX+1]:
            List_Connections = List_Connections + [[positionX-2, positionY, 1]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-2, positionY, 1]])), dim=0)
    if m-1 > positionY > 1:
        if S[positionY][positionX] == S[positionY-1][positionX] == S[positionY-2][positionX] == S[positionY+1][positionX]:
            List_Connections = List_Connections + [[positionX, positionY-2, 3]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY-2, 3]])), dim=0)
    if n-1 > positionX > 1 and m-1 > positionY > 1:
        if S[positionY][positionX] == S[positionY-1][positionX-1] == S[positionY-2][positionX-2] == S[positionY+1][positionX+1]:
            List_Connections = List_Connections + [[positionX-2, positionY-2, 2]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-2, positionY-2, 2]])), dim=0)
    if n-2 > positionX > 0 and m-1 > positionY > 1:
        if S[positionY][positionX] == S[positionY-1][positionX+1] == S[positionY-2][positionX+2] == S[positionY+1][positionX-1]:
            List_Connections = List_Connections + [[positionX+2, positionY-2, 4]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX+2, positionY-2, 4]])), dim=0)

    return(List_Connections)

class Board:
    def __init__(self):
        self.board_state = torch.zeros(6, 7).tolist()
        self.available_moves = [0,1,2,3,4,5,6]
        self.List_Connections_1 = []
        self.List_Connections_2 = []
    '''
    def to(self, device):
        """Moves all internal tensors to the specified device."""
        self.board_state = self.board_state.to(device)
        self.available_moves = self.available_moves.to(device)
        self.List_Connections_1 = self.List_Connections_1.to(device)
        self.List_Connections_2 = self.List_Connections_2.to(device)
        # Return self to allow for chaining: obj = TensorContainer(...).to(device)
        return self
    '''
    def move(self, player, position):
        S = self.board_state
        A_M = self.available_moves
        LC1 = self.List_Connections_1
        LC2 = self.List_Connections_2
        if position not in A_M:
            print('position not available')
            L = [1]
            for i in range(2): # Isso aqui é só pra dar um erro quando o movimento é ilegal
                L[i] = 1
        else:
            # Atualizando o tabuleiro
            for i in range(6):
                if S[6-i-1][position] == 0:
                    if player == 1:
                        S[6-i-1][position] = 1
                        self.List_Connections_1 = self.List_Connections_1 + Check_Conections(position, 6-i-1, S) 
                    else:
                        S[6-i-1][position] = -1
                        self.List_Connections_2 = self.List_Connections_2 + Check_Conections(position, 6-i-1, S)
                    if i == 5: # Atualizando a lista de movimentos disponiveis
                        A_M.remove(position)
                    break
             
            
            self.board_state = S
            self.available_moves = A_M
                

In [3]:
'''          
Tabuleiro = Board()
print(Tabuleiro.board_state)
print(Tabuleiro.available_moves)
Terminou = False
'''

NV1 = 0
NV2 = 0
NV0 = 0

LPV = []
Lx = []
for k in range(1):
    if k > 0 and k%100 == 0:
        print(k, NV1/(NV1+NV2+NV0), NV0/(NV1+NV2+NV0))
    
    
    Tabuleiro = Board()
    Terminou = False
    for i in range(7):
        for j in range(6):
            print(i, j)
            Jogadas_possiveis = Tabuleiro.available_moves
            if j%2 == 1:
                Tabuleiro.move(2, Jogadas_possiveis[rnd_N(len(Jogadas_possiveis))])
            else:
                '''jogada = int(input('Jogada: '))
                Tabuleiro.move(1, jogada)'''
                Tabuleiro.move(1, Jogadas_possiveis[rnd_N(len(Jogadas_possiveis))])
            print(Tabuleiro.board_state)
            print(Tabuleiro.available_moves)
            print(Tabuleiro.List_Connections_1)
            print(Tabuleiro.List_Connections_2)
            
            
            #print(Tabuleiro.board_state)
            #print(Tabuleiro.available_moves)
            #print(Tabuleiro.List_Connections_1)
            #print(Tabuleiro.List_Connections_2)
            '''
            if len(Tabuleiro.List_Connections_1) > 0:
                #print('vitória do jogador 1')
                NV1 = NV1 + 1
                Terminou = True
                break
            if len(Tabuleiro.List_Connections_2) > 0:
                #print('vitória do jogador 2')
                NV2 = NV2 + 1
                Terminou = True
                break'''
        if Terminou == True:
            LPV = LPV + [NV1/(NV1+NV2)]
            Lx = Lx + [k]
            break
    if Terminou == False:
        #print(Tabuleiro.board_state)
        NV0 = NV0 + 1

0 0
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]]
[0, 1, 2, 3, 4, 5, 6]
[]
[]
0 1
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1, 1, 0.0, 0.0, 0.0, 0.0, 0.0]]
[0, 1, 2, 3, 4, 5, 6]
[]
[]
0 2
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1, 1, 0.0, 0.0, 1, 0.0, 0.0]]
[0, 1, 2, 3, 4, 5, 6]
[]
[]
0 3
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1, 1, 0.0, 0.0, 1, 0.0, 0.0]]
[0, 1, 2, 3, 4, 5, 6]
[]

In [37]:
'''Depois a gente tem que ver quais tensores require grad e quais não, pra ficar mais eficiente'''
                
def minimax_pruned(Current_board_object, Model, depth, Player, alpha, beta): # Player = {1, != 1}
    Current_State = Current_board_object.board_state
    Current_Available_Moves = Current_board_object.available_moves
    if len(Current_board_object.List_Connections_1) > 0:
        return 1, None, True
    if len(Current_board_object.List_Connections_2) > 0:
        return 0, None, True
    elif len(Current_Available_Moves) == 0:
        return 0.5, None, True
    elif depth == 0:
        with torch.inference_mode():
            return Model(torch.tensor(Current_State)).item(), None, False

    else:
        if Player == 1: # Player 1 is the maximizer
            End_Node = True
            best_move = Current_Available_Moves[rnd_N(len(Current_Available_Moves))]
            Current_Evaluation = 0 # We start by assuming the worst possible case for player 1
            
            for move in Current_Available_Moves:
                child_board = Board()
                child_board.board_state = copy.deepcopy(Current_State)
                child_board.available_moves = copy.deepcopy(Current_Available_Moves)
                child_board.move(1, move)
                Evaluation_of_child, a, b = minimax_pruned(child_board, Model, depth-1, 2, alpha, beta)
                if Evaluation_of_child == 1:
                    if b == True:
                        return 1, move, True
                    else:
                        return 1, move, False
                if Evaluation_of_child > Current_Evaluation:
                    End_Node = False
                    Current_Evaluation = Evaluation_of_child
                    best_move = move
                alpha = max(alpha, Current_Evaluation)
                if beta <= alpha:
                    break
        else:
            End_Node = True
            Current_Evaluation = 1 # We start by assuming the worst possible case for player 2
            best_move = Current_Available_Moves[rnd_N(len(Current_Available_Moves))]
            
            for move in Current_Available_Moves:
                child_board = Board()
                child_board.board_state = copy.deepcopy(Current_State)
                child_board.available_moves = copy.deepcopy(Current_Available_Moves)
                child_board.move(2, move)
                Evaluation_of_child,  a, b = minimax_pruned(child_board, Model, depth-1, 1, alpha, beta)
                if Evaluation_of_child == 0:
                    if b == True:
                        return 0, move, True
                    else:
                        return 0, move, False
                if Evaluation_of_child < Current_Evaluation:
                    End_Node = False
                    Current_Evaluation = Evaluation_of_child
                    best_move = move
                beta = min(beta, Current_Evaluation)
                if beta <= alpha:
                    break
        
        
        return Current_Evaluation, best_move, End_Node              

In [38]:
class Model_Connect_4(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer_1 = nn.Linear(in_features=m*n, out_features=100)
        self.layer_2 = nn.Linear(in_features=100, out_features=100) 
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
        # Return the output of layer_2, a single feature, the same shape as y
        return torch.squeeze(self.sigmoid(self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x.reshape(-1, m*n)))))))) # computation goes through layer_1 first then the output of layer_1 goes through layer_2

Player_Model = Model_Connect_4().to(device)


In [17]:

DeltaT=0
for j in range(100):
            
    Tabuleiro2 = Board()
    
    for i in range(rnd_N(25)):
        Jogadas_possiveis = Tabuleiro2.available_moves
        Tabuleiro2.move((-1)**i, Jogadas_possiveis[rnd_N(len(Jogadas_possiveis))])
    #print(torch.tensor(Tabuleiro2.board_state))
    #print(Tabuleiro2.List_Connections_1, Tabuleiro2.List_Connections_2)
    
    t2 = time.time()
    #print(t2-t1)
    '''
    Avaliação_4, a = minimax_pruned(Tabuleiro2, Player_Model, 6, 1, 0, 1)
    
    if Avaliação_4 != 1 and Avaliação_4 != 0:
        Avaliação_6, b = minimax_pruned(Tabuleiro2, Player_Model, 7, 1, 0, 1)
        if Avaliação_6 == 1 or Avaliação_6 == 0: 
            print(torch.tensor(Tabuleiro2.board_state))
            print(a, b)
    '''
    print('Resultado:', minimax_pruned(Tabuleiro2, Player_Model, 6, 1, 0, 1))
    t3 = time.time()
    DeltaT = DeltaT + (t3-t2)/100
    print(t3-t2)
    #if j%1 == 0:
        #print('       ', t3-t2)

print(DeltaT)

Resultado: (1, 0, True)
0.015715599060058594
Resultado: (1, None, True)
0.0
Resultado: (1, 0, True)
0.18004751205444336
Resultado: (0.5029366612434387, 4, False)
1.592331886291504
Resultado: (1, 3, True)
0.6075832843780518
Resultado: (0.5139991641044617, 3, False)
0.8201146125793457
Resultado: (1, 2, True)
0.2514221668243408
Resultado: (0.5162386298179626, 4, False)
0.6873314380645752
Resultado: (1, None, True)
0.0
Resultado: (1, 1, True)
0.043993234634399414
Resultado: (1, 0, True)
0.015727996826171875
Resultado: (1, 4, True)
0.28636717796325684
Resultado: (1, 0, True)
0.0
Resultado: (1, 1, True)
0.12716054916381836
Resultado: (0.5077800154685974, 0, False)
0.4096200466156006
Resultado: (0, None, True)
0.0
Resultado: (0.5077800154685974, 0, False)
0.43276333808898926
Resultado: (1, 4, True)
1.5703725814819336
Resultado: (0.5115171074867249, 1, False)
0.21190381050109863
Resultado: (0.5244140625, 6, False)
1.0834689140319824
Resultado: (1, 5, True)
0.17573857307434082
Resultado: (0.527

In [41]:
def training_Batch(N, Player_Model, depth, P_rnd):
    Batch_X = []
    Batch_Y = []
    for game in range(N):
        board = Board()
        for movement in range(m*n):
            A_M = board.available_moves
            Board_State = board.board_state

            if (movement+N)%2 == 0:
                player = 1
            else:
                player = 2

            State_to_append = copy.deepcopy(Board_State)

            Evaluation, best_move, End_node = minimax_pruned(board, Player_Model, depth, player, 0, 1)
            if End_node == True:
                Batch_X.append(State_to_append)
                Batch_Y.append(Evaluation) 

            if rnd.random() > P_rnd:
                board.move(player, best_move)
            else:
                board.move(player, A_M[rnd_N(len(A_M))])
            #print(torch.tensor(Board_State))
            #print(Evaluation, best_move)
            if len(board.List_Connections_1) > 0 or len(board.List_Connections_2) > 0:
                break
            
        
        Batch_X.append(copy.deepcopy(board.board_state))
        
        
        if len(board.List_Connections_2) > 0:
            Result = torch.tensor(0).to(device)
        elif len(board.List_Connections_1) > 0:
            Result = torch.tensor(1).to(device)
        else:
            Result = torch.tensor(0.5).to(device)
        Batch_Y.append(Result)

        
    #return Batch_X, Batch_Y         
    return torch.tensor(Batch_X), torch.tensor(Batch_Y)

In [20]:
print(training_Batch(2, Player_Model, 5, 0.1))

(tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0., -1.,  0.,  0.,  0.],
         [-1.,  0.,  0., -1.,  0.,  0.,  0.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0., -1.,  0.,  0.,  0.],
         [-1.,  0.,  0., -1.,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  1., -1.,  1.,  0.]],

        [[ 0., -1.,  0., -1.,  0.,  0.,  0.],
         [ 0.,  1., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  1., -1., -1.,  0.,  0.,  0.],
         [-1., -1.,  1., -1.,  0.,  1.,  1.],
         [-1.,  1.,  1., -1.,  0.,  1., -1.],
         [ 1.,  1., -1.,  1., -1.,  1.,  1.]],

        [[ 0., -1.,  0., -1.,  0.,  0.,  0.],
         [ 0.,  1., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  1., -1., -1.,  0., -1.,  0.],
         [-1., -1.,  1., -1

In [62]:

Player_Model = Model_Connect_4().to(device)


In [99]:
def train(Player_Model, depth, Iterations, Epochs, Batch_Size, Print_Epoch_Loss):

    loss_function = nn.MSELoss()
    #optimizer = torch.optim.SGD(params = Player_Model.parameters(), lr = 1)
    optimizer = torch.optim.Adam(Player_Model.parameters(), lr=1e-3, betas=(0.9, 0.999))
    H=100
    for iteration in range(Iterations):
        if iteration%H == 0:
            print(iteration)
        training_data_X, training_data_Y = training_Batch(Batch_Size, Player_Model, depth, 0.5)
    
        for epoch in range(Epochs):
            
            y_pred = Player_Model(training_data_X.float())
            
            loss = loss_function(y_pred, training_data_Y.float())
            
            if epoch == 0 and iteration%H == 0: 
                print(loss)

            if epoch%Print_Epoch_Loss == 0:
                print(epoch, loss)
                
            if epoch == Epochs-1 and iteration%H == 0:
                print('      ', loss)
    
            optimizer.zero_grad()
        
            loss.backward()
        
            optimizer.step()


    return(Player_Model)
        

In [97]:
Player_Model = train(Player_Model, 3, 10, 5000, 1000, 500)
                    

0
tensor(0.0541, grad_fn=<MseLossBackward0>)
0 tensor(0.0541, grad_fn=<MseLossBackward0>)
500 tensor(0.0177, grad_fn=<MseLossBackward0>)
1000 tensor(0.0151, grad_fn=<MseLossBackward0>)
1500 tensor(0.0140, grad_fn=<MseLossBackward0>)
2000 tensor(0.0118, grad_fn=<MseLossBackward0>)
2500 tensor(0.0106, grad_fn=<MseLossBackward0>)
3000 tensor(0.0105, grad_fn=<MseLossBackward0>)
3500 tensor(0.0095, grad_fn=<MseLossBackward0>)
4000 tensor(0.0094, grad_fn=<MseLossBackward0>)
4500 tensor(0.0094, grad_fn=<MseLossBackward0>)
       tensor(0.0094, grad_fn=<MseLossBackward0>)
0 tensor(0.0496, grad_fn=<MseLossBackward0>)
500 tensor(0.0139, grad_fn=<MseLossBackward0>)
1000 tensor(0.0138, grad_fn=<MseLossBackward0>)
1500 tensor(0.0132, grad_fn=<MseLossBackward0>)
2000 tensor(0.0131, grad_fn=<MseLossBackward0>)
2500 tensor(0.0130, grad_fn=<MseLossBackward0>)
3000 tensor(0.0126, grad_fn=<MseLossBackward0>)
3500 tensor(0.0126, grad_fn=<MseLossBackward0>)
4000 tensor(0.0126, grad_fn=<MseLossBackward0>)
4

In [98]:
'''teste previsões modelo'''

training_data_X, training_data_Y = training_Batch(20, Player_Model, 2, 0.6)
y_pred = Player_Model(training_data_X.float())

print(torch.cat((training_data_Y.unsqueeze(-1), y_pred.unsqueeze(-1)), dim=1))
      

tensor([[1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.2138e-02],
        [0.0000e+00, 9.1768e-22],
        [1.0000e+00, 6.0881e-01],
        [1.0000e+00, 9.9783e-01],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 9.9999e-01],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 9.2161e-01],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.000

In [92]:
def play_against_model(Player_Model, depth, Human_player):
    New_game = True
    while New_game == True:
        New_game = input('New game? Y/N:')
        if New_game == 'y' or New_game == 'Y':
            New_game = True
        else:
            New_game = False
            break
            
        board = Board()
        #Human_player = rnd_N(2)+1
        for Turn in range(m*n):
            
            if Turn%2 == Human_player-1:               
                Valid_Move = False
                while Valid_Move == False:
                    Move = int(input('Your move:'))
                    if Move in board.available_moves:
                        Valid_Move = True
                    else:
                        print('Invalid move')
                board.move(Human_player, Move)
                print(torch.tensor(board.board_state))
                
                if len(board.List_Connections_1) > 0 or len(board.List_Connections_2) > 0:
                    print('You won!')
                    break
            
            else:
                Evaluation, best_move, End_node = minimax_pruned(board, Player_Model, depth, Turn%2+1, 0, 1)
                print(Evaluation, "Model' move:", best_move)
                board.move(Turn%2+1, best_move)
                print(torch.tensor(board.board_state))

                if len(board.List_Connections_1) > 0 or len(board.List_Connections_2) > 0:
                    print('The model won (=> vc é mt burro)')
                    break



In [93]:
play_against_model(Player_Model, 4, 2)

New game? Y/N: y


1 Model' move: 0
tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.]])


Your move: 4


tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0., -1.,  0.,  0.]])
1 Model' move: 0
tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0., -1.,  0.,  0.]])


Your move: 0


...

Your move: 6


tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  1.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  1.,  1.,  0.],
        [ 1.,  0.,  0., -1., -1., -1.,  0.],
        [ 1.,  0.,  0., -1., -1.,  1., -1.],
        [ 1.,  0.,  0., -1.,  1., -1., -1.]])
0 Model' move: 3
tensor([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  1.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  1.,  1.,  0.],
        [ 1.,  0.,  0., -1., -1., -1.,  0.],
        [ 1.,  0.,  0., -1., -1.,  1., -1.],
        [ 1.,  0.,  0., -1.,  1., -1., -1.]])


Your move: 6


tensor([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  1.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  1.,  1.,  0.],
        [ 1.,  0.,  0., -1., -1., -1., -1.],
        [ 1.,  0.,  0., -1., -1.,  1., -1.],
        [ 1.,  0.,  0., -1.,  1., -1., -1.]])
You won!


KeyboardInterrupt: Interrupted by user